**Mongo DB Notebook**

In [57]:
import pymongo
import pprint
import datetime
from bson.json_util import dumps
from bson.objectid import ObjectId
from bson.dbref import DBRef


client = pymongo.MongoClient("mongodb://127.0.0.1:27017")
db = client["distributed-db"]

In [40]:
# list could be used 
def print_cursor(cursor):
    for doc in cursor:
        pprint.pprint(doc)

In [41]:
# 1
db.products.insert_many([
    {
        "category" : "Phone",
        "model" : "iPhone 6",
        "producer" : "Apple",
        "price" : 600
    },
    {
        "category" : "TV",
        "model" : "SmartTV",
        "producer" : "Samsung",
        "diagonal": "100",
        "price" : 6000
    },
    {
        "category" : "Watch",
        "model" : "IWatch",
        "producer" : "Apple",
        "size": "m",
        "price" : 300,
    },
    {
        "category" : "Speaker",
        "model" : "Sound2",
        "producer" : "JBL",
        "Color": "Red",
        "price" : 100,
    },
    {
        "category" : "Laptop",
        "model" : "Super",
        "producer" : "Dell",
        "CPU": "i100",
        "GPU": "NVidia",
        "price" : 1000,
    },
])
print_cursor(
    db.products.find({})
)

{'_id': ObjectId('603b8cd70b05665191db5d67'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'price': 600,
 'producer': 'Apple'}
{'_id': ObjectId('603b8cd70b05665191db5d68'),
 'category': 'TV',
 'diagonal': '100',
 'model': 'SmartTV',
 'price': 6000,
 'producer': 'Samsung'}
{'_id': ObjectId('603b8cd70b05665191db5d69'),
 'category': 'Watch',
 'model': 'IWatch',
 'price': 300,
 'producer': 'Apple',
 'size': 'm'}
{'Color': 'Red',
 '_id': ObjectId('603b8cd70b05665191db5d6a'),
 'category': 'Speaker',
 'model': 'Sound2',
 'price': 100,
 'producer': 'JBL'}
{'CPU': 'i100',
 'GPU': 'NVidia',
 '_id': ObjectId('603b8cd70b05665191db5d6b'),
 'category': 'Laptop',
 'model': 'Super',
 'price': 1000,
 'producer': 'Dell'}


In [42]:
# 2 Напишіть запит, який виводіть усі товари (відображення у JSON)
cursor = db.products.find({})
list_cur = list(cursor)
dumps(list_cur)

'[{"_id": {"$oid": "603b8cd70b05665191db5d67"}, "category": "Phone", "model": "iPhone 6", "producer": "Apple", "price": 600}, {"_id": {"$oid": "603b8cd70b05665191db5d68"}, "category": "TV", "model": "SmartTV", "producer": "Samsung", "diagonal": "100", "price": 6000}, {"_id": {"$oid": "603b8cd70b05665191db5d69"}, "category": "Watch", "model": "IWatch", "producer": "Apple", "size": "m", "price": 300}, {"_id": {"$oid": "603b8cd70b05665191db5d6a"}, "category": "Speaker", "model": "Sound2", "producer": "JBL", "Color": "Red", "price": 100}, {"_id": {"$oid": "603b8cd70b05665191db5d6b"}, "category": "Laptop", "model": "Super", "producer": "Dell", "CPU": "i100", "GPU": "NVidia", "price": 1000}]'

In [43]:
# 3 Підрахуйте скільки товарів у певної категорії
db.products.count_documents({"category": "Laptop"})

1

In [132]:
# 4 Підрахуйте скільки є різних категорій товарів
len(db.products.distinct("category"))

5

In [45]:
# 5 Виведіть список всіх виробників товарів без повторів
db.products.distinct("producer")

['Apple', 'Dell', 'JBL', 'Samsung']

In [46]:
# 6
# Напишіть запити, які вибирають товари за різними критеріям і їх сукупності: 

#   категорія та ціна (в проміжку) - конструкція $and, 
print_cursor(
    db.products.find({"$and": [{"price": {"$gt": 300}}, {"price": {"$lt": 5000}}]})
)

{'_id': ObjectId('603b8cd70b05665191db5d67'),
 'category': 'Phone',
 'model': 'iPhone 6',
 'price': 600,
 'producer': 'Apple'}
{'CPU': 'i100',
 'GPU': 'NVidia',
 '_id': ObjectId('603b8cd70b05665191db5d6b'),
 'category': 'Laptop',
 'model': 'Super',
 'price': 1000,
 'producer': 'Dell'}


In [47]:
#   модель чи одна чи інша - конструкція $or,
print_cursor(
    db.products.find({"$or": [{"price": 300}, {"price": 50}]})
)

{'_id': ObjectId('603b8cd70b05665191db5d69'),
 'category': 'Watch',
 'model': 'IWatch',
 'price': 300,
 'producer': 'Apple',
 'size': 'm'}


In [48]:
#   виробники з переліку - конструкція $in
print_cursor(
    db.products.find({"producer": {"$in": ["JBL", "Samsung"]}})
)

{'_id': ObjectId('603b8cd70b05665191db5d68'),
 'category': 'TV',
 'diagonal': '100',
 'model': 'SmartTV',
 'price': 6000,
 'producer': 'Samsung'}
{'Color': 'Red',
 '_id': ObjectId('603b8cd70b05665191db5d6a'),
 'category': 'Speaker',
 'model': 'Sound2',
 'price': 100,
 'producer': 'JBL'}


In [49]:
# 7
# Оновить певні товари, змінивши існуючі значення і додайте нові 
# властивості (характеристики) усім товарам за певним критерієм
db.products.update_many(
        {"producer": "Apple"},
        {
        "$set": {"expensive": True, "producer": "AppleCalifornia"}
        }
    )
print_cursor(
    db.products.find({"producer": "AppleCalifornia"})
)

{'_id': ObjectId('603b8cd70b05665191db5d67'),
 'category': 'Phone',
 'expensive': True,
 'model': 'iPhone 6',
 'price': 600,
 'producer': 'AppleCalifornia'}
{'_id': ObjectId('603b8cd70b05665191db5d69'),
 'category': 'Watch',
 'expensive': True,
 'model': 'IWatch',
 'price': 300,
 'producer': 'AppleCalifornia',
 'size': 'm'}


In [50]:
# 8
# Знайдіть товари у яких є (присутнє поле) певні властивості
print_cursor(
    db.products.find({"expensive": {"$exists": True}})
)

{'_id': ObjectId('603b8cd70b05665191db5d67'),
 'category': 'Phone',
 'expensive': True,
 'model': 'iPhone 6',
 'price': 600,
 'producer': 'AppleCalifornia'}
{'_id': ObjectId('603b8cd70b05665191db5d69'),
 'category': 'Watch',
 'expensive': True,
 'model': 'IWatch',
 'price': 300,
 'producer': 'AppleCalifornia',
 'size': 'm'}


In [51]:
#  9
# Для знайдених товарів збільшіть їх вартість на певну суму
db.products.update_many(
        {"price": {"$gt": 900}},
        {"$inc": {"price": 50}}
    )
print_cursor(
    db.products.find({"price": {"$gt": 900}})
)

{'_id': ObjectId('603b8cd70b05665191db5d68'),
 'category': 'TV',
 'diagonal': '100',
 'model': 'SmartTV',
 'price': 6050,
 'producer': 'Samsung'}
{'CPU': 'i100',
 'GPU': 'NVidia',
 '_id': ObjectId('603b8cd70b05665191db5d6b'),
 'category': 'Laptop',
 'model': 'Super',
 'price': 1050,
 'producer': 'Dell'}


In [52]:
print_cursor(
    db.products.find({})
)

{'_id': ObjectId('603b8cd70b05665191db5d67'),
 'category': 'Phone',
 'expensive': True,
 'model': 'iPhone 6',
 'price': 600,
 'producer': 'AppleCalifornia'}
{'_id': ObjectId('603b8cd70b05665191db5d68'),
 'category': 'TV',
 'diagonal': '100',
 'model': 'SmartTV',
 'price': 6050,
 'producer': 'Samsung'}
{'_id': ObjectId('603b8cd70b05665191db5d69'),
 'category': 'Watch',
 'expensive': True,
 'model': 'IWatch',
 'price': 300,
 'producer': 'AppleCalifornia',
 'size': 'm'}
{'Color': 'Red',
 '_id': ObjectId('603b8cd70b05665191db5d6a'),
 'category': 'Speaker',
 'model': 'Sound2',
 'price': 100,
 'producer': 'JBL'}
{'CPU': 'i100',
 'GPU': 'NVidia',
 '_id': ObjectId('603b8cd70b05665191db5d6b'),
 'category': 'Laptop',
 'model': 'Super',
 'price': 1050,
 'producer': 'Dell'}


In [60]:
# Товари ви додаєте в замовлення - orders, яке містити вартість, ім'я замовника, і адресу доставки.
# Товари (items) в замовленні (order) повинні бути представлені у вигляді references, 
# а замовник (customer) у вигляді embed

# 1
# Створіть кілька замовлень з різними наборами товарів, 
# але так щоб один з товарів був у декількох замовленнях
db.orders.insert_many([
    {
        "order_number" : 201513,
        "date" : datetime.datetime(2015, 4, 14),
        "total_sum" : 6650,
        "customer" : {
            "name" : "Andrii",
            "surname" : "Petrovich",
            "phones" : [ 9876543, 1234567],
            "address" : "PTI, Peremohy 37, Kyiv, UA"
    },
    "payment" : {
        "card_owner" : "Andrii Petrovich",
        "cardId" : 12345678
    },
    "order_items_id" : [
        DBRef("products", "603b8cd70b05665191db5d67"),
        DBRef("products", "603b8cd70b05665191db5d68")
    ]
    },
    {
        "order_number" : 201514,
        "date" : datetime.datetime(2016, 4, 14),
        "total_sum" : 400,
        "customer" : {
            "name" : "Ivan",
            "surname" : "Ivanov",
            "phones" : [ 98716543, 123456437],
            "address" : "Lviv, Svobody 27"
    },
    "payment" : {
        "card_owner" : "Ivan Ivanov",
        "cardId" : 1234567899
    },
    "order_items_id" : [
        DBRef("products", "603b8cd70b05665191db5d69"),
        DBRef("products", "603b8cd70b05665191db5d6a")
    ]
    },
    {
        "order_number" : 201515,
        "date" : datetime.datetime(2017, 4, 14),
        "total_sum" : 1450,
        "customer" : {
            "name" : "Viktor",
            "surname" : "Radyu",
            "phones" : [ 987622543, 12345367],
            "address" : "Ostrog, Kiyvska 25/1"
    },
    "payment" : {
        "card_owner" : "Viktor Radyu",
        "cardId" : 1123456
    },
    "order_items_id" : [
        DBRef("products", "603b8cd70b05665191db5d69"),
        DBRef("products", "603b8cd70b05665191db5d6a"),
        DBRef("products", "603b8cd70b05665191db5d6b")
    ]
    },

])

In [61]:
# 2
# Виведіть всі замовлення
print_cursor(
    db.orders.find({})
)

{'_id': ObjectId('603b90d10b05665191db5d74'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Petrovich'},
 'date': datetime.datetime(2015, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d67'),
                    DBRef('products', '603b8cd70b05665191db5d68')],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Petrovich'},
 'total_sum': 6650}
{'_id': ObjectId('603b90d10b05665191db5d75'),
 'customer': {'address': 'Lviv, Svobody 27',
              'name': 'Ivan',
              'phones': [98716543, 123456437],
              'surname': 'Ivanov'},
 'date': datetime.datetime(2016, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a')],
 'order_number': 201514,
 'payment': {'cardId': 1234567899, 'card_owner': 'Ivan Ivanov'},
 'total_sum':

In [62]:
# 3
# Виведіть замовлення з вартістю більше певного значення
print_cursor(
    db.orders.find({"total_sum": {"$gt": 1400}})
)

{'_id': ObjectId('603b90d10b05665191db5d74'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Petrovich'},
 'date': datetime.datetime(2015, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d67'),
                    DBRef('products', '603b8cd70b05665191db5d68')],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Petrovich'},
 'total_sum': 6650}
{'_id': ObjectId('603b90d10b05665191db5d76'),
 'customer': {'address': 'Ostrog, Kiyvska 25/1',
              'name': 'Viktor',
              'phones': [987622543, 12345367],
              'surname': 'Radyu'},
 'date': datetime.datetime(2017, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a'),
                    DBRef('products', '603b8cd70b05665191db5d6b')],
 'order_number': 201515,
 'paym

In [63]:
# 4
# Знайдіть замовлення зроблені одним замовником
print_cursor(
    db.orders.find({"customer.name": "Andrii", "customer.surname": "Petrovich"})
)

{'_id': ObjectId('603b90d10b05665191db5d74'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Petrovich'},
 'date': datetime.datetime(2015, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d67'),
                    DBRef('products', '603b8cd70b05665191db5d68')],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Petrovich'},
 'total_sum': 6650}


In [117]:
# 5
# Знайдіть всі замовлення з певним товаром (товарами) (шукати можна по ObjectId)
product = db.products.find_one({"category": "Watch"})
print_cursor(
    db.orders.find({"order_items_id": DBRef("products", str(product["_id"]))})
)

{'_id': ObjectId('603b90d10b05665191db5d75'),
 'customer': {'address': 'Lviv, Svobody 27',
              'name': 'Ivan',
              'phones': [98716543, 123456437],
              'surname': 'Ivanov'},
 'date': datetime.datetime(2016, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a')],
 'order_number': 201514,
 'payment': {'cardId': 1234567899, 'card_owner': 'Ivan Ivanov'},
 'total_sum': 400}
{'_id': ObjectId('603b90d10b05665191db5d76'),
 'customer': {'address': 'Ostrog, Kiyvska 25/1',
              'name': 'Viktor',
              'phones': [987622543, 12345367],
              'surname': 'Radyu'},
 'date': datetime.datetime(2017, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a'),
                    DBRef('products', '603b8cd70b05665191db5d6b')],
 'order_number': 201515,
 'payment': {'cardId':

In [122]:
# 6
# Додайте в усі замовлення з певним товаром ще один товар і 
# збільште існуючу вартість замовлення на деяке значення Х
new_product = db.products.find_one({"model": "iPhone 6"})
product = db.products.find_one({"category": "Watch"})
db.orders.update_many(
    {"order_items_id": DBRef("products", str(product["_id"]))},
    {"$inc": {"total_sum": new_product["price"]},
     "$addToSet": {"order_items_id": [DBRef("products", str(new_product["_id"]))]}
    }
)

print_cursor(
    db.orders.find({"order_items_id": DBRef("products", str(product["_id"]))})
)

{'_id': ObjectId('603b90d10b05665191db5d75'),
 'customer': {'address': 'Lviv, Svobody 27',
              'name': 'Ivan',
              'phones': [98716543, 123456437],
              'surname': 'Ivanov'},
 'date': datetime.datetime(2016, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a'),
                    [DBRef('products', '603b8cd70b05665191db5d67')]],
 'order_number': 201514,
 'payment': {'cardId': 1234567899, 'card_owner': 'Ivan Ivanov'},
 'total_sum': 1000}
{'_id': ObjectId('603b90d10b05665191db5d76'),
 'customer': {'address': 'Ostrog, Kiyvska 25/1',
              'name': 'Viktor',
              'phones': [987622543, 12345367],
              'surname': 'Radyu'},
 'date': datetime.datetime(2017, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a'),
                    DBRef('products', '603b8c

In [137]:
#  7
# Виведіть кількість товарів в певному замовленні
print_cursor(
    db.orders.aggregate([
        {"$match": {"order_number": 201515}},
        {"$project": { "numberOfItems": {"$size": "$order_items_id"}}}
    ])
)

{'_id': ObjectId('603b90d10b05665191db5d76'), 'numberOfItems': 4}


In [139]:
# 8
# Виведіть тільки інформацію про кастомера і номери кредитної карт, 
# для замовлень вартість яких перевищує певну суму
print_cursor(
    db.orders.aggregate([
        {"$match": {"total_sum": {"$gt": 1400}}},
        {"$project": { "_id": False, "customer": 1, "cardId": "$payment.cardId"}}
    ])
)

{'cardId': 12345678,
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Petrovich'}}
{'cardId': 1123456,
 'customer': {'address': 'Ostrog, Kiyvska 25/1',
              'name': 'Viktor',
              'phones': [987622543, 12345367],
              'surname': 'Radyu'}}


In [150]:
#  9 
# Видаліть товар з замовлень, зроблених за певний період дат
db.orders.update_many(
    {"$and": [{"date": {"$gt": datetime.datetime(2013, 4, 13)}}, {"date": {"$lt": datetime.datetime(2016, 1, 14)}}]},
    {"$set": {"order_items_id": []}}
)

print_cursor(
    db.orders.find({"$and": [{"date": {"$gt": datetime.datetime(2013, 4, 13)}}, {"date": {"$lt": datetime.datetime(2016, 1, 14)}}]})
)

{'_id': ObjectId('603b90d10b05665191db5d74'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'Petrovich'},
 'date': datetime.datetime(2015, 4, 14, 0, 0),
 'order_items_id': [],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Petrovich'},
 'total_sum': 6650}


In [152]:
# 10
# Перейменуйте у всіх замовлення ім'я (прізвище) замовника
db.orders.update_many(
    {},
    {"$set": {"customer.surname": "NewSurname"}}
)

print_cursor(
    db.orders.find()
)

{'_id': ObjectId('603b90d10b05665191db5d74'),
 'customer': {'address': 'PTI, Peremohy 37, Kyiv, UA',
              'name': 'Andrii',
              'phones': [9876543, 1234567],
              'surname': 'NewSurname'},
 'date': datetime.datetime(2015, 4, 14, 0, 0),
 'order_items_id': [],
 'order_number': 201513,
 'payment': {'cardId': 12345678, 'card_owner': 'Andrii Petrovich'},
 'total_sum': 6650}
{'_id': ObjectId('603b90d10b05665191db5d75'),
 'customer': {'address': 'Lviv, Svobody 27',
              'name': 'Ivan',
              'phones': [98716543, 123456437],
              'surname': 'NewSurname'},
 'date': datetime.datetime(2016, 4, 14, 0, 0),
 'order_items_id': [DBRef('products', '603b8cd70b05665191db5d69'),
                    DBRef('products', '603b8cd70b05665191db5d6a'),
                    [DBRef('products', '603b8cd70b05665191db5d67')]],
 'order_number': 201514,
 'payment': {'cardId': 1234567899, 'card_owner': 'Ivan Ivanov'},
 'total_sum': 1000}
{'_id': ObjectId('603b90d10b056

In [176]:
# Створіть Сapped collection яка б містила 5 останніх відгуків на наш інтернет-магазин. 
# Структуру запису визначіть самостійно.
db.create_collection('comments', capped=True, max=5, size=1024)

Collection(Database(MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True), 'distributed-db'), 'comments')

In [180]:
# Перевірте що при досягненні обмеження старі відгуки будуть затиратись
db.comments.insert_many([
    {
        "author": "Ivan",
        "comment_text": "Good thing"
    },
    {
        "author": "Slava",
        "comment_text": "Super bad thing"
    },
    {
        "author": "Victor",
        "comment_text": "Nice!"
    },
    {
        "author": "John",
        "comment_text": "Very very bad"
    },
    {
        "author": "Merry",
        "comment_text": "Perfect!!"
    },
])
print_cursor(
    db.comments.find({})
)

{'_id': ObjectId('603bc6fa0b05665191db5dba'),
 'author': 'Ivan',
 'comment_text': 'Good thing'}
{'_id': ObjectId('603bc6fa0b05665191db5dbb'),
 'author': 'Slava',
 'comment_text': 'Super bad thing'}
{'_id': ObjectId('603bc6fa0b05665191db5dbc'),
 'author': 'Victor',
 'comment_text': 'Nice!'}
{'_id': ObjectId('603bc6fa0b05665191db5dbd'),
 'author': 'John',
 'comment_text': 'Very very bad'}
{'_id': ObjectId('603bc6fa0b05665191db5dbe'),
 'author': 'Merry',
 'comment_text': 'Perfect!!'}


In [184]:
db.comments.insert_one ({
        "author": "Kate",
        "comment_text": "Brilliant"
    })
print_cursor(
    db.comments.find({})
)
db.comments.count_documents({})

{'_id': ObjectId('603bc6fa0b05665191db5dbb'),
 'author': 'Slava',
 'comment_text': 'Super bad thing'}
{'_id': ObjectId('603bc6fa0b05665191db5dbc'),
 'author': 'Victor',
 'comment_text': 'Nice!'}
{'_id': ObjectId('603bc6fa0b05665191db5dbd'),
 'author': 'John',
 'comment_text': 'Very very bad'}
{'_id': ObjectId('603bc6fa0b05665191db5dbe'),
 'author': 'Merry',
 'comment_text': 'Perfect!!'}
{'_id': ObjectId('603bc7040b05665191db5dbf'),
 'author': 'Kate',
 'comment_text': 'Brilliant'}


5